In [1]:
!nvidia-smi

Sat Dec 27 20:12:55 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   43C    P8             11W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [2]:
%pip install torch torchcodec transformers datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 41.4 MB/s eta 0:00:0000:01


## Probing CTC Models

Demonstrate Wav2Vec2's speech transcription capabilities

In [3]:
from datasets import load_dataset

dataset = load_dataset(
    "hf-internal-testing/librispeech_asr_dummy", "clean", split="validation"
)
dataset

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:104: UserWarning: 
Error while fetching `HF_TOKEN` secret value from your vault: 'Requesting secret HF_TOKEN timed out. Secrets can only be fetched when running from the Colab UI.'.
You are not authenticated with the Hugging Face Hub in this notebook.
If the error persists, please let us know by opening an issue on GitHub (https://github.com/huggingface/huggingface_hub/issues/new).
  warnings.warn(


Dataset({
    features: ['file', 'audio', 'text', 'speaker_id', 'chapter_id', 'id'],
    num_rows: 73
})

Inspect the audio sample as well as the transcription

In [4]:
from IPython.display import Audio

sample = dataset[2]

print(sample["text"])
Audio(sample["audio"]["array"], rate=sample["audio"]["sampling_rate"])

HE TELLS US THAT AT THIS FESTIVE SEASON OF THE YEAR WITH CHRISTMAS AND ROAST BEEF LOOMING BEFORE US SIMILES DRAWN FROM EATING AND ITS RESULTS OCCUR MOST READILY TO THE MIND


Load the wav2vec2 checkpoint

In [5]:
from transformers import pipeline

pipe = pipeline("automatic-speech-recognition", model="facebook/wav2vec2-base-100h")

config.json: 0.00B [00:00, ?B/s]

pytorch_model.bin:   0%|          | 0.00/378M [00:00<?, ?B/s]

Some weights of the model checkpoint at facebook/wav2vec2-base-100h were not used when initializing Wav2Vec2ForCTC: ['wav2vec2.mask_time_emb_vector']
- This IS expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-base-100h and are newly initialized: ['wav2vec2.masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tokenizer_config.json:   0%|          | 0.00/376 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/358 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/378M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/96.0 [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/159 [00:00<?, ?B/s]

Device set to use cuda:0


Pass a copy of the data so we can safely reuse the audio sample

In [9]:
audio_tensor = sample["audio"].get_all_samples()
audio_tensor = audio_tensor.data.numpy()

audio_array = audio_tensor.squeeze()


pipe(audio_array)

{'text': 'HE TELLS US THAT AT THIS FESTIVE SEASON OF THE YEAR WITH CHRISTMAUS AND ROSE BEEF LOOMING BEFORE US SIMALYIS DRAWN FROM EATING AND ITS RESULTS OCCUR MOST READILY TO THE MIND'}

CTCs are acoustic models and thus struggle with spelling, along with casing and spelling.

Load whisper base on the GPU

In [33]:
import torch
from transformers import pipeline

device = "cuda:0" if torch.cuda.is_available() else "cpu"
pipe = pipeline(
    "automatic-speech-recognition", model="openai/whisper-base", device=device
)

Device set to use cuda:0


Generate a prediction with whisper but we have to specify `max_new_tokens` this time

In [34]:
pipe(sample["audio"], max_new_tokens=256)

ValueError: You have passed more than 3000 mel input features (> 30 seconds) which automatically enables long-form generation which requires the model to predict timestamp tokens. Please either pass `return_timestamps=True` or make sure to pass no more than 3000 mel input features.

Use multilingual librispeech to begin assessing Whisper's multilingual capabilities

In [13]:
dataset = load_dataset(
    "facebook/multilingual_librispeech", "spanish", split="test", streaming=True
)
sample = next(iter(dataset))

Resolving data files:   0%|          | 0/48 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/30 [00:00<?, ?it/s]

Inspect a sample and listen to the audio segment

In [16]:
print(sample["transcript"])
Audio(sample["audio"]["array"], rate=sample["audio"]["sampling_rate"])

y las almas buscando algún alivio se revuelven ansiosas y hacen el mundo que así resulta ser del dolor obra el dolor o la nada quien tenga corazón venga y escoja


Transcribe with the pipeline

In [30]:
audio_copy = sample

print(audio_copy["transcript"])
pipe(audio_copy["audio"], return_timestamps=True, max_new_tokens=256, generate_kwargs={"task": "translate"})

y las almas buscando algún alivio se revuelven ansiosas y hacen el mundo que así resulta ser del dolor obra el dolor o la nada quien tenga corazón venga y escoja


Whisper did not predict an ending timestamp, which can happen if audio is cut off in the middle of a word. Also make sure WhisperTimeStampLogitsProcessor was used during generation.


{'text': " I thought you had found me, I thought you had found me, I thought you had found me, I thought you had found me, I thought you had found me, I thought you had found me, I thought you had found me, I thought you had found me, I thought you had found me, I thought you had found me, I thought you had found me, I thought you had found me, I thought you had found me, I thought you had found me, I thought you had found me, I thought you had found me, I thought you had found me, I thought you had found me, I thought you had found me, I thought you had found me, I thought you had found me, I thought you had found me, I thought you had found me, I thought you had found me, I thought you had found me, I thought you had found me, I thought you had found me, I thought you had found me, I thought you had found me, I thought you had found me, I thought you had found me, I thought you had found me, I thought you had found me, I thought you had found me, I thought you had found me, I thought

Deal with the very pressing issue of the 30 second mel spectrogram conversion problem...

In [35]:
import numpy as np

target_length_in_m = 5

# convert from minutes to seconds (* 60) to num samples (* sampling rate)
sampling_rate = pipe.feature_extractor.sampling_rate
target_length_in_samples = target_length_in_m * 60 * sampling_rate

# iterate over our streaming dataset, concatenating samples until we hit our target
long_audio = []
for sample in dataset:
    long_audio.extend(sample["audio"]["array"])
    if len(long_audio) > target_length_in_samples:
        break

long_audio = np.asarray(long_audio)

# how did we do?
seconds = len(long_audio) / 16000
minutes, seconds = divmod(seconds, 60)
print(f"Length of audio sample is {minutes} minutes {seconds:.2f} seconds")

Length of audio sample is 5.0 minutes 15.00 seconds


Now we tell transformers we want to batch our audio sample into 8 distinct audio samples (with small overlap for reconstruction), and each chunk should be 30s in length

In [36]:
pipe(
    long_audio,
    max_new_tokens=256,
    generate_kwargs={"task": "transcribe"},
    chunk_length_s=30,
    batch_size=8,
)

Using `chunk_length_s` is very experimental with seq2seq models. The results will not necessarily be entirely accurate and will have caveats. More information: https://github.com/huggingface/transformers/pull/20104. Ignore this warning with pipeline(..., ignore_warning=True). To use Whisper for long-form transcription, use rather the model's `generate` method directly as the model relies on it's own chunking mechanism (cf. Whisper original paper, section 3.8. Long-form Transcription).


{'text': " I thought you were fun. I thought you were fun. I thought you were fun. I thought you were fun. I thought you were fun. I thought you were fun. I thought you were fun. I thought you were fun. I thought you were fun. I thought you were fun. I thought you were fun. I thought you were fun. I thought you were fun. I thought you were fun. I thought you were fun. I thought you were fun. I thought you were fun. I thought you were fun. I thought you were fun. I thought you were fun. I thought you were fun. I thought you were fun. I thought you were fun. I thought you were fun. I thought you were fun. I thought you were fun. I thought you were fun. I thought you were fun. I thought you were fun. I thought you were fun. I thought you were fun. I thought you were fun. I thought you were fun. I thought you were fun. I thought you were fun. I thought you were fun. I thought you were fun. I thought you were fun. I thought you were fun. I thought you were fun. I thought you were fun. I tho

Try to activate timestamps

In [37]:
pipe(
    long_audio,
    max_new_tokens=256,
    generate_kwargs={"task": "transcribe"},
    chunk_length_s=30,
    batch_size=8,
    return_timestamps=True,
)["chunks"]

Using `chunk_length_s` is very experimental with seq2seq models. The results will not necessarily be entirely accurate and will have caveats. More information: https://github.com/huggingface/transformers/pull/20104. Ignore this warning with pipeline(..., ignore_warning=True). To use Whisper for long-form transcription, use rather the model's `generate` method directly as the model relies on it's own chunking mechanism (cf. Whisper original paper, section 3.8. Long-form Transcription).


[{'timestamp': (106.0, 109.0),
  'text': " I thought you had found me, I thought you had found me, I thought you had found me, I thought you had found me, I thought you had found me, I thought you had found me, I thought you had found me, I thought you had found me, I thought you had found me, I thought you had found me, I thought you had found me, I thought you had found me, I thought you had found me, I thought you had found me, I thought you had found me, I thought you had found me, I thought you had found me, I thought you had found me, I thought you had found me, I thought you had found me, I thought you had found me, I thought you had found me, I thought you had found me, I thought you had found me, I thought you had found me, I thought you had found me, I thought you had found me, I thought you had found me, I thought you had found me, I thought you had found me, I thought you had found me, I thought you had found me, I thought you had found me, I thought you had found me, I tho